# filter useful field
```outpath="/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/user_flow"```

1. 8613851651044		用户账号/手机号
2. 1388170568.241000		记录时间，等于流结束时间，UTC格式
3. q1.qlogo.cn/g?b=mqq&k=sauicrjKpJO73QXibiaYOUfWA&t=1371003426&refer=mqq&s=100 url 
4. q1.qlogo.cn		WEB/WAP请求HOST
5. http			承载协议类型
6. 1781        连接时长
7. 4           大类业务ID
8. 1035        小类业务ID


In [107]:
!head -n 1 /media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/data/userurl/gn/cm/jiangsu/nanjing/20131228/39server/userurl_20131228_235801_001039.log

In [108]:
filePath='/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/data/userurl/gn/cm/jiangsu/nanjing/20131228/*/*'
# filePath='/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/data/userurl/gn/cm/jiangsu/nanjing/20131228/39server/userurl_20131228_173539_001039.log'

In [109]:
outpath="/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/user_flow"

In [110]:
lines = sc.textFile(filePath)
parts = lines.map(lambda l: l.split("\t")).filter(lambda line : len(line)==88)

In [111]:
def map_function(p):
    if p[38] == '-':
        p[38] =''
    p[38]=p[38].split(p[35])[-1].split(':')[0]
    p[35]=p[35].split(':')[0]
    return (p[4], [p[0],p[35]+p[38],p[35], p[64],int(p[46]),p[59],p[60]])
#     if p[38].startswith("http")
user_flow=parts.map(lambda  p: map_function(p))
user_flow.count()

979984173

In [112]:
# import shutil
# if os.path.exists(outpath):
#     shutil.rmtree(outpath)
user_flow.saveAsTextFile(outpath)

# get category

In [113]:
# category domain
outPath="/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw/result_jw_new/categroy/domain_category"
category=user_flow.map(lambda  p: (p[1][2],p[1][5]))
category=category.distinct()

import shutil 
if os.path.exists(outPath):
    shutil.rmtree(outPath)
category.saveAsTextFile(outPath)

In [114]:
# category url    
    
outPath="/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw/result_jw_new/categroy/url_category"
category=user_flow.map(lambda  p: (p[1][1],p[1][5]))
category=category.distinct()

import shutil 
if os.path.exists(outPath):
    shutil.rmtree(outPath)
category.saveAsTextFile(outPath)

# domain2vec data prepare

In [6]:
user_flow = sc.textFile('/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/user_flow/*').map(lambda t : eval(t))

In [7]:
domain_flow=user_flow.map(lambda u :(u[0],[u[1][0],u[1][2]]))
# def add(a, b): 
#     a.append(b)
#     return a 
# def add(a, b): return a + str(b)
user_domain_flow=domain_flow.groupByKey().mapValues(list)
user_domain_flow.saveAsTextFile('/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/user_domain_flow')

In [3]:
# url_flow=user_flow.map(lambda u :(u[0],[u[1][0],u[1][1]]))
# def add(a, b): 
#     a.append(b)
#     return a 
# # def add(a, b): return a + str(b)
# user_url_flow=url_flow.foldByKey([],add).repartition(600)
# user_url_flow.saveAsTextFile('/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/user_url_flow')

## to get domain seq (user local python)
* input: 

```('8615157868499', [['1388199497.030000', 'loc.map.baidu.com'], ['1388199474.678000', 'client.map.baidu.com']])```
* output:

```client.map.baidu.com loc.map.baidu.com```

In [44]:
input_path="/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/user_domain_flow"
# output_path='/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/domain_data_repeat'
output_path='/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/domain_data_remove_api'

In [45]:
def join_distinct_list(li):
    str=''
    for i,domain in enumerate(li):
        if i==0:
            str=str+li[i]+' '
        elif li[i]==li[i-1]:
            pass
        else:
            str=str+li[i]+' '
    return str.strip()


for fname in os.listdir(input_path):
    if fname.startswith("part"):
        fi=open(os.path.join(input_path, fname), mode='r',encoding="utf-8") 
        for line in fi.readlines():
            line=tuple(eval(line))
            if len(line)!=2:
                continue
            try:
                tmp=sorted(line[1],key=lambda l: float(str(l[0])))
            except:
                pass
            with open(os.path.join(output_path, fname), mode='a', encoding='utf-8') as fo:
#               
                res = []
                pre_time = 0.0
                for i in tmp:
#                     print(i)
#                     break
                    if float(i[0]) - float(pre_time) > 0.4:
                        res.append(i[1])
                    pre_time = float(i[0])
#                 fo.write(' '.join(res)+'\n')
                fo.write(join_distinct_list(res)+'\n')
        fi.close()

In [46]:
input_path="/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/user_domain_flow"
# output_path='/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/domain_data_repeat'
output_path='/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/domain_data_adduser_removeapi'

In [47]:
def join_distinct_list(li):
    str=''
    for i,domain in enumerate(li):
        if i==0:
            str=str+li[i]+' '
        elif li[i]==li[i-1]:
            pass
        else:
            str=str+li[i]+' '
    return str.strip()


for fname in os.listdir(input_path):
    if fname.startswith("part"):
        fi=open(os.path.join(input_path, fname), mode='r',encoding="utf-8") 
        for line in fi.readlines():
            line=tuple(eval(line))
            if len(line)!=2:
                continue
            try:
                tmp=sorted(line[1],key=lambda l: float(str(l[0])))
            except:
                pass
            with open(os.path.join(output_path, fname), mode='a', encoding='utf-8') as fo:
#               
                res = [line[0]]
                pre_time = 0.0
                for i in tmp:
#                     print(i)
#                     break
                    if float(i[0]) - float(pre_time) > 0.4:
                        res.append(i[1])
                    pre_time = float(i[0])
#                 fo.write(' '.join(res)+'\n')
                fo.write(join_distinct_list(res)+'\n')
        fi.close()

In [37]:
a = sc.textFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/domain_data_remove_api/part-00000")
b = sc.textFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/domain_data_remove_api/0.4")

In [38]:
a = set(a.flatMap(lambda l : l.split()).collect())
b = set(b.flatMap(lambda l : l.split()).collect())

In [39]:
len(a)

1620

In [40]:
len(b)

1520

In [41]:
len(a - b)

100

In [36]:
a - b 

{'102.imagebam.com',
 '103.imagebam.com',
 '105.imagebam.com',
 '111.30.130.109',
 '112.25.58.157',
 '112.25.8.149',
 '119.75.220.29',
 '120.132.153.234',
 '120.204.205.183',
 '139.panguso.com',
 '149.img.pp.sohu.com.cn',
 '221.176.31.140',
 '3g.v.qq.com',
 '42.51.12.71',
 '61.4.185.48',
 'a-m-s.poco.cn',
 'a1.itc.cn',
 'a167.photo.store.qq.com',
 'a254.photo.store.qq.com',
 'a255.photo.store.qq.com',
 'a269.photo.store.qq.com',
 'a330.photo.store.qq.com',
 'a5.shuqireader.com',
 'active.book.qq.com',
 'adslist.wandoujia.com',
 'aia.entrust.net',
 'ajax.aspnetcdn.com',
 'allot.im.hupan.com',
 'android.clients.google.com',
 'api.52youtu.com',
 'api.device.xiaomi.net',
 'api.ibaozou.com',
 'api.letv.com',
 'app.api.autohome.com.cn',
 'app.api.gexin.channel.push.mcp.weibo.cn',
 'app.jd.com',
 'athena.simba.taobao.com',
 'b122.photo.store.qq.com',
 'b261.photo.store.qq.com',
 'b330.photo.store.qq.com',
 'bank-51.ads.mp.mydas.mobi',
 'bank-57.ads.mp.mydas.mobi',
 'bank-74.ads.mp.mydas.mobi'

In [12]:
a.count()

43390


## train a word2vec
* input: domain_data

```client.map.baidu.com loc.map.baidu.com```
* output:model-word2vec

```monitor.uu.qq.com -0.003459 -0.001384 -0.003676 0.002139 -0.004722 0.000682 0.002843 -0.001901 0.003471 0.002348 -0.000193 -0.003921 0.000865 0.004497 -0.003894 0.004958 0.001414 0.004182 -0.001125 -0.004657 0.000835 0.004116 -0.000095 -0.003578 -0.003982 0.003971 0.003855 0.002501 0.002137 -0.003359 0.002720 -0.003433 -0.000297 0.000109 0.002659 0.002966 -0.000975 -0.001353 0.004007 0.003263 0.003226 -0.001994 0.002135 -0.004421 0.003325 0.000021 0.000243 0.002711 -0.002844 -0.001345 -0.004296 0.001295 0.002124 -0.002261 -0.004481 -0.002293 -0.003195 -0.001187 0.003280 -0.003042 0.003973 0.003177 -0.000498 -0.000378 -0.000753 -0.004273 0.002578 0.001137 0.002161 0.000618 0.000697 -0.004914 -0.004877 0.002786 0.000019 0.004735 0.001274 -0.002370 -0.002219 -0.000442 -0.004202 0.001722 0.001433 -0.000560 -0.002201 -0.003625 0.001559 -0.002486 -0.000846 -0.001618 -0.002898 0.000449 -0.004796 0.004013 -0.001226 -0.004375 -0.004110 0.003417 -0.004163 0.000714```

In [1]:
 # import modules & set up logging 
 import gensim, logging
 import os
 logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

sentences = MySentences('/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/domain_data') # a memory-friendly iterator

/home/jw/anaconda3/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [16]:
import shutil 
for i in [100,200,500,1000,2000]:
    print("train-model-size%s"%str(i))
    model = gensim.models.Word2Vec(sentences,size=i, window=9, min_count=8, workers=500)
    output_model_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-size"+str(i)
    if not os.path.exists(output_model_path):
        os.mkdir(output_model_path)
    model.save_word2vec_format("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-size%s/model-all-word2vec"%str(i), fvocab="/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-size%s/model-all-vocabulary"%str(i), binary=False)
    

2016-12-12 09:07:45,436 : INFO : collecting all words and their counts
2016-12-12 09:07:45,460 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


train-model-size100


2016-12-12 09:07:46,827 : INFO : PROGRESS: at sentence #10000, processed 1899223 words, keeping 17112 word types
2016-12-12 09:07:47,639 : INFO : PROGRESS: at sentence #20000, processed 3748470 words, keeping 23844 word types
2016-12-12 09:07:48,460 : INFO : PROGRESS: at sentence #30000, processed 5667588 words, keeping 28558 word types
2016-12-12 09:07:49,274 : INFO : PROGRESS: at sentence #40000, processed 7553817 words, keeping 32508 word types
2016-12-12 09:07:50,132 : INFO : PROGRESS: at sentence #50000, processed 9450799 words, keeping 36533 word types
2016-12-12 09:07:50,946 : INFO : PROGRESS: at sentence #60000, processed 11257893 words, keeping 39899 word types
2016-12-12 09:07:51,796 : INFO : PROGRESS: at sentence #70000, processed 13164779 words, keeping 42860 word types
2016-12-12 09:07:52,641 : INFO : PROGRESS: at sentence #80000, processed 15074664 words, keeping 45710 word types
2016-12-12 09:07:53,439 : INFO : PROGRESS: at sentence #90000, processed 16912958 words, keep

train-model-size200


2016-12-12 09:56:10,884 : INFO : PROGRESS: at sentence #10000, processed 1899223 words, keeping 17112 word types
2016-12-12 09:56:11,839 : INFO : PROGRESS: at sentence #20000, processed 3748470 words, keeping 23844 word types
2016-12-12 09:56:12,861 : INFO : PROGRESS: at sentence #30000, processed 5667588 words, keeping 28558 word types
2016-12-12 09:56:13,858 : INFO : PROGRESS: at sentence #40000, processed 7553817 words, keeping 32508 word types
2016-12-12 09:56:14,838 : INFO : PROGRESS: at sentence #50000, processed 9450799 words, keeping 36533 word types
2016-12-12 09:56:15,721 : INFO : PROGRESS: at sentence #60000, processed 11257893 words, keeping 39899 word types
2016-12-12 09:56:16,709 : INFO : PROGRESS: at sentence #70000, processed 13164779 words, keeping 42860 word types
2016-12-12 09:56:17,722 : INFO : PROGRESS: at sentence #80000, processed 15074664 words, keeping 45710 word types
2016-12-12 09:56:18,707 : INFO : PROGRESS: at sentence #90000, processed 16912958 words, keep

...

2016-12-12 10:42:24,053 : INFO : PROGRESS: at sentence #10000, processed 1899223 words, keeping 17112 word types
2016-12-12 10:42:25,012 : INFO : PROGRESS: at sentence #20000, processed 3748470 words, keeping 23844 word types
2016-12-12 10:42:25,991 : INFO : PROGRESS: at sentence #30000, processed 5667588 words, keeping 28558 word types
2016-12-12 10:42:26,998 : INFO : PROGRESS: at sentence #40000, processed 7553817 words, keeping 32508 word types
2016-12-12 10:42:27,974 : INFO : PROGRESS: at sentence #50000, processed 9450799 words, keeping 36533 word types
2016-12-12 10:42:28,913 : INFO : PROGRESS: at sentence #60000, processed 11257893 words, keeping 39899 word types
2016-12-12 10:42:29,862 : INFO : PROGRESS: at sentence #70000, processed 13164779 words, keeping 42860 word types
2016-12-12 10:42:30,773 : INFO : PROGRESS: at sentence #80000, processed 15074664 words, keeping 45710 word types
2016-12-12 10:42:31,655 : INFO : PROGRESS: at sentence #90000, processed 16912958 words, keep

train-model-size1000


2016-12-12 11:39:28,261 : INFO : PROGRESS: at sentence #10000, processed 1899223 words, keeping 17112 word types
2016-12-12 11:39:29,219 : INFO : PROGRESS: at sentence #20000, processed 3748470 words, keeping 23844 word types
2016-12-12 11:39:30,166 : INFO : PROGRESS: at sentence #30000, processed 5667588 words, keeping 28558 word types
2016-12-12 11:39:31,124 : INFO : PROGRESS: at sentence #40000, processed 7553817 words, keeping 32508 word types
2016-12-12 11:39:32,110 : INFO : PROGRESS: at sentence #50000, processed 9450799 words, keeping 36533 word types
2016-12-12 11:39:33,048 : INFO : PROGRESS: at sentence #60000, processed 11257893 words, keeping 39899 word types
2016-12-12 11:39:34,032 : INFO : PROGRESS: at sentence #70000, processed 13164779 words, keeping 42860 word types
2016-12-12 11:39:35,018 : INFO : PROGRESS: at sentence #80000, processed 15074664 words, keeping 45710 word types
2016-12-12 11:39:35,952 : INFO : PROGRESS: at sentence #90000, processed 16912958 words, keep

train-model-size2000


2016-12-12 13:06:05,385 : INFO : PROGRESS: at sentence #10000, processed 1899223 words, keeping 17112 word types
2016-12-12 13:06:06,241 : INFO : PROGRESS: at sentence #20000, processed 3748470 words, keeping 23844 word types
2016-12-12 13:06:07,175 : INFO : PROGRESS: at sentence #30000, processed 5667588 words, keeping 28558 word types
2016-12-12 13:06:08,120 : INFO : PROGRESS: at sentence #40000, processed 7553817 words, keeping 32508 word types
2016-12-12 13:06:09,042 : INFO : PROGRESS: at sentence #50000, processed 9450799 words, keeping 36533 word types
2016-12-12 13:06:09,868 : INFO : PROGRESS: at sentence #60000, processed 11257893 words, keeping 39899 word types
2016-12-12 13:06:10,758 : INFO : PROGRESS: at sentence #70000, processed 13164779 words, keeping 42860 word types
2016-12-12 13:06:11,622 : INFO : PROGRESS: at sentence #80000, processed 15074664 words, keeping 45710 word types
2016-12-12 13:06:12,510 : INFO : PROGRESS: at sentence #90000, processed 16912958 words, keep

In [ ]:
import shutil 
for i in [2,5,10,20,50,80]:
    print("train-model-size%s"%str(i))
    model = gensim.models.Word2Vec(sentences,size=i, window=9, min_count=8, workers=500)
    output_model_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-size"+str(i)
    if not os.path.exists(output_model_path):
        os.mkdir(output_model_path)
    model.save_word2vec_format("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-size%s/model-all-word2vec"%str(i), fvocab="/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-size%s/model-all-vocabulary"%str(i), binary=False)
    

2016-12-15 17:40:04,521 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2016-12-15 17:40:04,523 : INFO : collecting all words and their counts


train-model-size2


2016-12-15 17:40:08,562 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-12-15 17:40:27,951 : INFO : PROGRESS: at sentence #10000, processed 1899223 words, keeping 17112 word types
2016-12-15 17:40:37,763 : INFO : PROGRESS: at sentence #20000, processed 3748470 words, keeping 23844 word types
2016-12-15 17:40:55,585 : INFO : PROGRESS: at sentence #30000, processed 5667588 words, keeping 28558 word types
2016-12-15 17:41:09,369 : INFO : PROGRESS: at sentence #40000, processed 7553817 words, keeping 32508 word types


In [ ]:
import shutil 
for i in range(5,20):
    print("train-model-size%s"%str(i))
    model = gensim.models.Word2Vec(sentences,size=i, window=9, min_count=8, workers=500)
    output_model_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-size"+str(i)
    if not os.path.exists(output_model_path):
        os.mkdir(output_model_path)
    model.save_word2vec_format("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-size%s/model-all-word2vec"%str(i), fvocab="/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-size%s/model-all-vocabulary"%str(i), binary=False)
    

2016-12-18 21:41:33,088 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2016-12-18 21:41:33,089 : INFO : collecting all words and their counts
2016-12-18 21:41:33,197 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


train-model-size5


2016-12-18 21:41:35,748 : INFO : PROGRESS: at sentence #10000, processed 1899223 words, keeping 17112 word types
2016-12-18 21:41:37,182 : INFO : PROGRESS: at sentence #20000, processed 3748470 words, keeping 23844 word types
2016-12-18 21:41:38,316 : INFO : PROGRESS: at sentence #30000, processed 5667588 words, keeping 28558 word types
2016-12-18 21:41:39,475 : INFO : PROGRESS: at sentence #40000, processed 7553817 words, keeping 32508 word types
2016-12-18 21:41:40,513 : INFO : PROGRESS: at sentence #50000, processed 9450799 words, keeping 36533 word types
2016-12-18 21:41:41,658 : INFO : PROGRESS: at sentence #60000, processed 11257893 words, keeping 39899 word types
2016-12-18 21:41:42,770 : INFO : PROGRESS: at sentence #70000, processed 13164779 words, keeping 42860 word types
2016-12-18 21:41:43,961 : INFO : PROGRESS: at sentence #80000, processed 15074664 words, keeping 45710 word types
2016-12-18 21:41:45,466 : INFO : PROGRESS: at sentence #90000, processed 16912958 words, keep

In [ ]:
# import modules & set up logging 
import gensim, logging
import os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

sentences = MySentences('/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/domain_data_repeat/') # a memory-friendly iterator


import shutil 
for i in [2,5,10,15,20,50,80,100,200,500,1000]:
    print("train-repeat-model-size%s"%str(i))
    model = gensim.models.Word2Vec(sentences,size=i, window=9, min_count=8, workers=500)
    output_model_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/repeat-model-size"+str(i)
    if not os.path.exists(output_model_path):
        os.mkdir(output_model_path)
    model.save_word2vec_format("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/repeat-model-size%s/model-all-word2vec"%str(i), fvocab="/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/repeat-model-size%s/model-all-vocabulary"%str(i), binary=False)
    

/home/jw/anaconda3/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
2016-12-20 20:58:27,965 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2016-12-20 20:58:27,965 : INFO : collecting all words and their counts
2016-12-20 20:58:27,995 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


train-repeat-model-size2


2016-12-20 20:58:30,150 : INFO : PROGRESS: at sentence #10000, processed 3568607 words, keeping 17112 word types
2016-12-20 20:58:32,122 : INFO : PROGRESS: at sentence #20000, processed 6955625 words, keeping 23844 word types
2016-12-20 20:58:33,784 : INFO : PROGRESS: at sentence #30000, processed 10412539 words, keeping 28558 word types
2016-12-20 20:58:35,411 : INFO : PROGRESS: at sentence #40000, processed 13806782 words, keeping 32508 word types
2016-12-20 20:58:37,464 : INFO : PROGRESS: at sentence #50000, processed 17311582 words, keeping 36533 word types
2016-12-20 20:58:39,350 : INFO : PROGRESS: at sentence #60000, processed 20659148 words, keeping 39899 word types
2016-12-20 20:58:41,259 : INFO : PROGRESS: at sentence #70000, processed 24160762 words, keeping 42860 word types
2016-12-20 20:58:43,154 : INFO : PROGRESS: at sentence #80000, processed 27626580 words, keeping 45710 word types
2016-12-20 20:58:44,920 : INFO : PROGRESS: at sentence #90000, processed 31006334 words, k

In [ ]:

# import modules & set up logging 
import gensim, logging
import os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname

    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()

sentences = MySentences('/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/domain_data/') # a memory-friendly iterator


import shutil 
for i in [1,2,3,5,7,10,13,15,20,30,50]:
    print("train-repeat-model-window%s"%str(i))
    model = gensim.models.Word2Vec(sentences,size=100, window=i, min_count=8, workers=500)
    output_model_path = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-window"+str(i)
    if not os.path.exists(output_model_path):
        os.mkdir(output_model_path)
    model.save_word2vec_format("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-window%s/model-all-word2vec"%str(i), fvocab="/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/result_jw_new/myline/model-window%s/model-all-vocabulary"%str(i), binary=False)

train-repeat-model-size1


2017-01-06 20:46:27,816 : INFO : collecting all words and their counts
2017-01-06 20:46:31,073 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-01-06 20:46:37,569 : INFO : PROGRESS: at sentence #10000, processed 1899223 words, keeping 17112 word types
2017-01-06 20:46:41,632 : INFO : PROGRESS: at sentence #20000, processed 3748470 words, keeping 23844 word types
